# Importing libraries

In [ ]:
import os, glob, json, csv, subprocess, sys, re
from git import *
from subprocess import Popen, PIPE
from os import path
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# Defining repository and directories

In [ ]:
userhome = os.path.expanduser("~")
repository = userhome + r"/different-diff/dataset/openjpa/"
analyze_dir = userhome + r'/different-diff/analyze/analyze_openjpa/'

# Loading the earliest affected versions dataset

In [ ]:
affected_version = pd.read_csv(analyze_dir + '05_finding_versions/01_affected_version/earliest_version.csv')
cols = ['bug_id','earliest_affected_version','date_release']
aff = affected_version[cols]
aff

# Combining buggyfile data with the earliest versions dataset

In [ ]:
algorithms = ['histogram','minimal','myers','patience']

In [ ]:
colname = ['bug_id','bugintro_commitID','parent_id','filepath','number_of_buggyline']

In [ ]:
#Combining data with earliest versions
for xx, algr in enumerate(algorithms):
    filedata = pd.read_csv(analyze_dir + '03_file-diff/03_number_of_files/' + algr + '_total_bugline.csv')
    filedata = filedata[colname]

    details = filedata.join(aff.set_index('bug_id')[['earliest_affected_version','date_release']], on='bug_id')
    details.to_csv(analyze_dir + '05_finding_versions/01_affected_version/affected_version_for_' + algr + '.csv')

In [ ]:
details

# Start comparing buglines with the blamed files

# Histogram

In [ ]:
filelist_hist = glob.glob(analyze_dir + '04_annotate/01_blame_files/blame_histogram/csv/*')
blamefiles_hist = ';'.join(filelist_hist).split(';')
blamefiles_hist.sort()
print ("Found " + str(len(blamefiles_hist)) + " files") 

In [ ]:
diff_files_hist = []
for files in glob.iglob(analyze_dir + "03_file-diff/02_buggylines/histogram/*"):
    fn = open(files, encoding="utf8", errors='ignore')
    fn = fn.read().split('\n')
    
    if "--- /dev/null" not in fn[0]:
        if fn[0] != '':
            diff_files_hist.append(files)
    else:
        pass
    
diff_files_hist.sort()
print ("Found " + str(len(diff_files_hist)) + " files")

In [ ]:
h = 850
print (blamefiles_hist[h])
print ("\n" + diff_files_hist[h])

In [ ]:
df_algo_files_hist = []
x = 0
while x < len(diff_files_hist):
    sys.stdout.write("\rExtracting file %i" %(x+1))
    sys.stdout.flush()
    df_algo_files_hist.append([diff_files_hist[x]])
    x += 1

df_algo_files_hist

In [ ]:
#Open csv file from result of git blame for <filename>.java
for num, blame in enumerate(blamefiles_hist):
    sys.stdout.write('\rComparing bugline_files and blame_files: %i' % (num+1) + ' out of %i' % (len(blamefiles_hist)))
    sys.stdout.flush()
    
    fields = ['line_number', 'code', 'date']
    blamefile = pd.read_csv(blame)
    blamefile = blamefile[fields]

    origin_code = []
    for n in range(0, len(blamefile)):
        temp = [blamefile.iloc[n][0], str(blamefile.iloc[n][1]).replace('nan',''), blamefile.iloc[n][2]]
        origin_code.append(temp)
    
    #Open diff_file <filename>.java-HBASE-xxxx_<diffalgorithm>buglines.diff vs blame_<filename>
    for m, df in enumerate(df_algo_files_hist[num]):
        dfname = re.search(("(?<=histogram\/)\w.*"), df)
        dfname = dfname.group(0)
        
        diff_file = open(analyze_dir + '03_file-diff/02_buggylines/histogram/' + dfname, encoding="utf8", errors='ignore')
        bugline = diff_file.read().split('\n')

        if bugline[len(bugline)-1] == '':
            for i in range(0,len(bugline)-1):
                bugline[i] = bugline[i][1:]
            del bugline[len(bugline)-1]
        else:
            for i in range(0,len(bugline)):
                bugline[i] = bugline[i][1:]
        
        bug_id = re.compile('(OPENJPA-[0-9]*)')
        bug_id = bug_id.findall(df)
        
        for vers in range(0,len(aff)):
            if bug_id[0] == aff.iloc[vers][0]:
                version = aff.iloc[vers][1]
                daterelease = aff.iloc[vers][2]
                break;
            else:
                version = 'None'
                daterelease = 'None'
        
        num = 0
        blame_compare = []
        org_code = [el[1] for el in origin_code]
        for y, bugs in enumerate(bugline):
            if bugs in org_code:
                for x in range(num, len(origin_code)):                
                    codex = origin_code[x][1]
                    if bugs == codex:
                        if version != 'None':
                            if origin_code[x][2] <= daterelease:
                                flag = 'bug-introducing change'
                            else:
                                flag = 'incorrect'
                        else:
                            flag = 'unknown'
                        temp = [bug_id[0], origin_code[x][0], bugs, "True", origin_code[x][2], version, daterelease, flag]
                        blame_compare.append(temp)
                        num = origin_code[x][0]
                        break;
            else:
                temp = [bug_id[0], 'None', bugs, "False", 'None', version, daterelease, 'unknown']
                blame_compare.append(temp)
                #break;
        
        fnames = re.search(("((?<=histogram\/\_).[a-zA-Z0-9-.]+(?=_))"), df)
        fnames = fnames.group(0)
        
        #algoname = re.search(("[_]\w+.diff"), df)
        #algoname = algoname.group(0)
        
        blamename = re.search(("(?<=\/csv\/)(?:\w.*)"), blame)
        blamename = blamename.group(0)
        
        with open (analyze_dir + r'04_annotate/02_diff-file_blame-file_comparison/histogram_comparison/diff_' + fnames + "_histogram-" + blamename, 'w') as csvfile:
            writers = csv.writer(csvfile, delimiter = ",")
            header = ['bug_id', 'line_number', 'buggy_code', 'does_bugline_from_originfile?', 'date_added_in_originfile', 'earliest_affected_version', 'version_release_date', 'bug_class']
            writers.writerow(header)
            for item in blame_compare:
                writers.writerow(item)
                
print ('\nComparing buglines are complete')

# Minimal

In [ ]:
filelist_min = glob.glob(analyze_dir + '04_annotate/01_blame_files/blame_minimal/csv/*')
blamefiles_min = ';'.join(filelist_min).split(';')
blamefiles_min.sort()
print ("Found " + str(len(blamefiles_min)) + " files")

In [ ]:
diff_files_min = []
for files in glob.iglob(analyze_dir + "03_file-diff/02_buggylines/minimal/*"):
    fn = open(files, encoding="utf8", errors='ignore')
    fn = fn.read().split('\n')
    
    if "--- /dev/null" not in fn[0]:
        if fn[0] != '':
            diff_files_min.append(files)
    else:
        pass
diff_files_min.sort()
print ("Found " + str(len(diff_files_min)) + " files")

In [ ]:
h = 345
print (blamefiles_min[h])
print ("\n" + diff_files_min[h])

In [ ]:
df_algo_files_min = []
x = 0
while x < len(diff_files_min):
    sys.stdout.write("\rExtracting file %i" %(x+1))
    sys.stdout.flush()
    df_algo_files_min.append([diff_files_min[x]])
    x += 1
    
df_algo_files_min

In [ ]:
#Open csv file from result of git blame for <filename>.java
for num, blame in enumerate(blamefiles_min):
    sys.stdout.write('\rComparing bugline_files and blame_files: %i' % (num+1) + ' out of %i' % (len(blamefiles_min)))
    sys.stdout.flush()
    
    fields = ['line_number', 'code', 'date']
    blamefile = pd.read_csv(blame)
    blamefile = blamefile[fields]

    origin_code = []
    for n in range(0, len(blamefile)):
        temp = [blamefile.iloc[n][0], str(blamefile.iloc[n][1]).replace('nan',''), blamefile.iloc[n][2]]
        origin_code.append(temp)
    
    #Open diff_file <filename>.java-HBASE-xxxx_<diffalgorithm>buglines.diff vs blame_<filename>
    for m, df in enumerate(df_algo_files_min[num]):
        dfname = re.search(("(?<=minimal\/)\w.*"), df)
        dfname = dfname.group(0)
        
        diff_file = open(analyze_dir + '03_file-diff/02_buggylines/minimal/' + dfname, encoding="utf8", errors='ignore')
        bugline = diff_file.read().split('\n')

        if bugline[len(bugline)-1] == '':
            for i in range(0,len(bugline)-1):
                bugline[i] = bugline[i][1:]
            del bugline[len(bugline)-1]
        else:
            for i in range(0,len(bugline)):
                bugline[i] = bugline[i][1:]
        
        bug_id = re.compile('(OPENJPA-[0-9]*)')
        bug_id = bug_id.findall(df)
        
        for vers in range(0,len(aff)):
            if bug_id[0] == aff.iloc[vers][0]:
                version = aff.iloc[vers][1]
                daterelease = aff.iloc[vers][2]
                break;
            else:
                version = 'None'
                daterelease = 'None'
        
        num = 0
        blame_compare = []
        org_code = [el[1] for el in origin_code]
        for y, bugs in enumerate(bugline):
            if bugs in org_code:
                for x in range(num, len(origin_code)):                
                    codex = origin_code[x][1]
                    if bugs == codex:
                        if version != 'None':
                            if origin_code[x][2] <= daterelease:
                                flag = 'bug-introducing change'
                            else:
                                flag = 'incorrect'
                        else:
                            flag = 'unknown'
                        temp = [bug_id[0], origin_code[x][0], bugs, "True", origin_code[x][2], version, daterelease, flag]
                        blame_compare.append(temp)
                        num = origin_code[x][0]
                        break;
            else:
                temp = [bug_id[0], 'None', bugs, "False", 'None', version, daterelease, 'unknown']
                blame_compare.append(temp)
                #break;
        
        fnames = re.search(("((?<=minimal\/\_).[a-zA-Z0-9-.]+(?=_))"), df)
        fnames = fnames.group(0)
        
        #algoname = re.search(("[_]\w+.diff"), df)
        #algoname = algoname.group(0)
        
        blamename = re.search(("(?<=\/csv\/)(?:\w.*)"), blame)
        blamename = blamename.group(0)
        
        with open (analyze_dir + r'04_annotate/02_diff-file_blame-file_comparison/minimal_comparison/diff_' + fnames + "_minimal-" + blamename, 'w') as csvfile:
            writers = csv.writer(csvfile, delimiter = ",")
            header = ['bug_id', 'line_number', 'buggy_code', 'does_bugline_from_originfile?', 'date_added_in_originfile', 'earliest_affected_version', 'version_release_date', 'bug_class']
            writers.writerow(header)
            for item in blame_compare:
                writers.writerow(item)
                
print ('\nComparing buglines are complete')

# Myers

In [ ]:
filelist_myers = glob.glob(analyze_dir + '04_annotate/01_blame_files/blame_myers/csv/*')
blamefiles_myers = ';'.join(filelist_myers).split(';')
blamefiles_myers.sort()
print ("Found " + str(len(blamefiles_myers)) + " files")

In [ ]:
diff_files_myers = []
for files in glob.iglob(analyze_dir + "03_file-diff/02_buggylines/myers/*"):
    fn = open(files, encoding="utf8", errors='ignore')
    fn = fn.read().split('\n')
    
    if "--- /dev/null" not in fn[0]:
        if fn[0] != '':
            diff_files_myers.append(files)
    else:
        pass
diff_files_myers.sort()
print ("Found " + str(len(diff_files_myers)) + " files")

In [ ]:
h = 24
print (blamefiles_myers[h])
print ("\n" + diff_files_myers[h])

In [ ]:
df_algo_files_myers = []
x = 0
while x < len(diff_files_myers):
    sys.stdout.write("\rExtracting file %i" %(x+1))
    sys.stdout.flush()
    df_algo_files_myers.append([diff_files_myers[x]])
    x += 1
    
df_algo_files_myers

In [ ]:
#Open csv file from result of git blame for <filename>.java
for num, blame in enumerate(blamefiles_myers):
    sys.stdout.write('\rComparing bugline_files and blame_files: %i' % (num+1) + ' out of %i' % (len(blamefiles_myers)))
    sys.stdout.flush()
    
    fields = ['line_number', 'code', 'date']
    blamefile = pd.read_csv(blame)
    blamefile = blamefile[fields]

    origin_code = []
    for n in range(0, len(blamefile)):
        temp = [blamefile.iloc[n][0], str(blamefile.iloc[n][1]).replace('nan',''), blamefile.iloc[n][2]]
        origin_code.append(temp)
    
    #Open diff_file <filename>.java-HBASE-xxxx_<diffalgorithm>buglines.diff vs blame_<filename>
    for m, df in enumerate(df_algo_files_myers[num]):
        dfname = re.search(("(?<=myers\/)\w.*"), df)
        dfname = dfname.group(0)
        
        diff_file = open(analyze_dir + '03_file-diff/02_buggylines/myers/' + dfname, encoding="utf8", errors='ignore')
        bugline = diff_file.read().split('\n')

        if bugline[len(bugline)-1] == '':
            for i in range(0,len(bugline)-1):
                bugline[i] = bugline[i][1:]
            del bugline[len(bugline)-1]
        else:
            for i in range(0,len(bugline)):
                bugline[i] = bugline[i][1:]
        
        bug_id = re.compile('(OPENJPA-[0-9]*)')
        bug_id = bug_id.findall(df)
        
        for vers in range(0,len(aff)):
            if bug_id[0] == aff.iloc[vers][0]:
                version = aff.iloc[vers][1]
                daterelease = aff.iloc[vers][2]
                break;
            else:
                version = 'None'
                daterelease = 'None'
        
        num = 0
        blame_compare = []
        org_code = [el[1] for el in origin_code]
        for y, bugs in enumerate(bugline):
            if bugs in org_code:
                for x in range(num, len(origin_code)):                
                    codex = origin_code[x][1]
                    if bugs == codex:
                        if version != 'None':
                            if origin_code[x][2] <= daterelease:
                                flag = 'bug-introducing change'
                            else:
                                flag = 'incorrect'
                        else:
                            flag = 'unknown'
                        temp = [bug_id[0], origin_code[x][0], bugs, "True", origin_code[x][2], version, daterelease, flag]
                        blame_compare.append(temp)
                        num = origin_code[x][0]
                        break;
            else:
                temp = [bug_id[0], 'None', bugs, "False", 'None', version, daterelease, 'unknown']
                blame_compare.append(temp)
                #break;
        
        fnames = re.search(("((?<=myers\/\_).[a-zA-Z0-9-.]+(?=_))"), df)
        fnames = fnames.group(0)
        
        #algoname = re.search(("[_]\w+.diff"), df)
        #algoname = algoname.group(0)
        
        blamename = re.search(("(?<=\/csv\/)(?:\w.*)"), blame)
        blamename = blamename.group(0)
        
        with open (analyze_dir + r'04_annotate/02_diff-file_blame-file_comparison/myers_comparison/diff_' + fnames + "_myers-" + blamename, 'w') as csvfile:
            writers = csv.writer(csvfile, delimiter = ",")
            header = ['bug_id', 'line_number', 'buggy_code', 'does_bugline_from_originfile?', 'date_added_in_originfile', 'earliest_affected_version', 'version_release_date', 'bug_class']
            writers.writerow(header)
            for item in blame_compare:
                writers.writerow(item)
                
print ('\nComparing buglines are complete')

# Patience

In [ ]:
filelist_pat = glob.glob(analyze_dir + '04_annotate/01_blame_files/blame_patience/csv/*')
blamefiles_pat = ';'.join(filelist_pat).split(';')
blamefiles_pat.sort()
print ("Found " + str(len(blamefiles_pat)) + " files")

In [ ]:
diff_files_pat = []
for files in glob.iglob(analyze_dir + "03_file-diff/02_buggylines/patience/*"):
    fn = open(files, encoding="utf8", errors='ignore')
    fn = fn.read().split('\n')
    
    if "--- /dev/null" not in fn[0]:
        if fn[0] != '':
            diff_files_pat.append(files)
    else:
        pass
diff_files_pat.sort()
print ("Found " + str(len(diff_files_pat)) + " files")

In [ ]:
h = 601
print (blamefiles_pat[h])
print ("\n" + diff_files_pat[h])

In [ ]:
df_algo_files_pat = []
x = 0
while x < len(diff_files_pat):
    sys.stdout.write("\rExtracting file %i" %(x+1))
    sys.stdout.flush()
    df_algo_files_pat.append([diff_files_pat[x]])
    x += 1
    
df_algo_files_pat

In [ ]:
#Open csv file from result of git blame for <filename>.java
for num, blame in enumerate(blamefiles_pat):
    sys.stdout.write('\rComparing bugline_files and blame_files: %i' % (num+1) + ' out of %i' % (len(blamefiles_pat)))
    sys.stdout.flush()
    
    fields = ['line_number', 'code', 'date']
    blamefile = pd.read_csv(blame)
    blamefile = blamefile[fields]

    origin_code = []
    for n in range(0, len(blamefile)):
        temp = [blamefile.iloc[n][0], str(blamefile.iloc[n][1]).replace('nan',''), blamefile.iloc[n][2]]
        origin_code.append(temp)
    
    #Open diff_file <filename>.java-HBASE-xxxx_<diffalgorithm>buglines.diff vs blame_<filename>
    for m, df in enumerate(df_algo_files_pat[num]):
        dfname = re.search(("(?<=patience\/)\w.*"), df)
        dfname = dfname.group(0)
        
        diff_file = open(analyze_dir + '03_file-diff/02_buggylines/patience/' + dfname, encoding="utf8", errors='ignore')
        bugline = diff_file.read().split('\n')

        if bugline[len(bugline)-1] == '':
            for i in range(0,len(bugline)-1):
                bugline[i] = bugline[i][1:]
            del bugline[len(bugline)-1]
        else:
            for i in range(0,len(bugline)):
                bugline[i] = bugline[i][1:]
        
        bug_id = re.compile('(OPENJPA-[0-9]*)')
        bug_id = bug_id.findall(df)
        
        for vers in range(0,len(aff)):
            if bug_id[0] == aff.iloc[vers][0]:
                version = aff.iloc[vers][1]
                daterelease = aff.iloc[vers][2]
                break;
            else:
                version = 'None'
                daterelease = 'None'
        
        num = 0
        blame_compare = []
        org_code = [el[1] for el in origin_code]
        for y, bugs in enumerate(bugline):
            if bugs in org_code:
                for x in range(num, len(origin_code)):                
                    codex = origin_code[x][1]
                    if bugs == codex:
                        if version != 'None':
                            if origin_code[x][2] <= daterelease:
                                flag = 'bug-introducing change'
                            else:
                                flag = 'incorrect'
                        else:
                            flag = 'unknown'
                        temp = [bug_id[0], origin_code[x][0], bugs, "True", origin_code[x][2], version, daterelease, flag]
                        blame_compare.append(temp)
                        num = origin_code[x][0]
                        break;
            else:
                temp = [bug_id[0], 'None', bugs, "False", 'None', version, daterelease, 'unknown']
                blame_compare.append(temp)
                #break;
        
        fnames = re.search(("((?<=patience\/\_).[a-zA-Z0-9-.]+(?=_))"), df)
        fnames = fnames.group(0)
        
        #algoname = re.search(("[_]\w+.diff"), df)
        #algoname = algoname.group(0)
        
        blamename = re.search(("(?<=\/csv\/)(?:\w.*)"), blame)
        blamename = blamename.group(0)
        
        with open (analyze_dir + r'04_annotate/02_diff-file_blame-file_comparison/patience_comparison/diff_' + fnames + "_patience-" + blamename, 'w') as csvfile:
            writers = csv.writer(csvfile, delimiter = ",")
            header = ['bug_id', 'line_number', 'buggy_code', 'does_bugline_from_originfile?', 'date_added_in_originfile', 'earliest_affected_version', 'version_release_date', 'bug_class']
            writers.writerow(header)
            for item in blame_compare:
                writers.writerow(item)
                
print ('\nComparing buglines are complete')